## Langchain expression language

In [29]:
from dotenv import load_dotenv
import os
load_dotenv()

# langsmith tracking
#https://smith.langchain.com/
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
#groqAPIkey = os.getenv('GROQ_API_KEY')

In [30]:
#!pip install langchain_groq

In [31]:
from langchain_groq import ChatGroq

LLM=ChatGroq(model='gemma2-9b-it', temperature= 0.1)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI engineer expert."),
    ("user", "Question: {que}") 
])
chain = prompt|LLM
chain.invoke({'que':"what is generativeAI"})

AIMessage(content="As an AI engineer, I can tell you that **Generative AI** is a fascinating and rapidly evolving field within artificial intelligence. \n\nHere's a breakdown:\n\n**What it is:**\n\nGenerative AI refers to a type of artificial intelligence that focuses on creating new content. This content can take many forms, including:\n\n* **Text:**  Articles, stories, poems, dialogue, code\n* **Images:** Photos, artwork, illustrations\n* **Audio:** Music, sound effects, speech\n* **Video:** Animations, short films\n* **Other data:** 3D models, synthetic datasets\n\n**How it works:**\n\nGenerative AI models are typically trained on massive datasets of existing content. They learn the underlying patterns and structures within this data, allowing them to generate new content that resembles the training data. \n\nCommon techniques used in generative AI include:\n\n* **Generative Adversarial Networks (GANs):**  Two neural networks compete against each other. One network generates content

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
messages = [
    SystemMessage(content="Translate the following from english to french"),
    HumanMessage(content="hello, how are you?")
]

LLM.invoke(messages)

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 27, 'total_tokens': 55, 'completion_time': 0.055472449, 'prompt_time': 0.002578508, 'queue_time': None, 'total_time': 0.058050957}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9e88f3b-3732-41de-837c-5b54cb02e728-0', usage_metadata={'input_tokens': 27, 'output_tokens': 28, 'total_tokens': 55})

In [11]:
# LCEL ----> chain the component together

chain = LLM|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, ça va ? \n'

In [ ]:
# Prompts template
from langchain_core.prompts import ChatPromptTemplate

genericTemplate= "Convert  the following into {DBName} format."

few_shot_dict = [
    {"question": "how to calculate total employees from table A",
     "answer": "The SQL query: 'SELECT COUNT(DISTINCT *) FROM table_A;'"},

    {"question": "how to find average salary from table B",
     "answer": "The SQL query: 'SELECT AVG(salary) FROM table_B;'"} ,

     {"question": "Hi or hello",
     "answer": "How can i help you sirg? "}
]

# Convert dictionary to list of HumanMessage and AIMessage
few_shot_examples = []
for example in few_shot_dict:
    few_shot_examples.append(HumanMessage(content=example['question']))
    few_shot_examples.append(AIMessage(content=example['answer']))

prompt = ChatPromptTemplate.from_messages([
        ("system","""You are the natural language to SQL convertor translate the. {genericTemplate} so be mindful.
          I have provided you some examples also.
          If user question is not related to SQL then don't show any random response just say 'Don't know sir' """),
        *few_shot_examples,
        ("user","User question is: {text}. Give the sql query int the provided format.")
    ]
) 


# prompt.invoke({"DBName":"Mysql workbench","text":"how are you bro"})
# chain = prompt|LLM |parser
# chain.invoke({"DBName":"Mysql workbench","text":"how many department are there?"})


In [24]:
prompt.invoke({"DBName":"Mysql workbench","text":"how are you bro"})

ChatPromptValue(messages=[SystemMessage(content='You are the natural language to SQL convertor translate the."+genericTemplate +" so be mindful.\n          I have provided you some examples also.\n          If user question is not related to SQL then don\'t show any random response just say \'Don\'t know sir\' '), HumanMessage(content='how to calculate total employees from table A'), AIMessage(content="The SQL query: 'SELECT COUNT(DISTINCT *) FROM table_A;'"), HumanMessage(content='how to find average salary from table B'), AIMessage(content="The SQL query: 'SELECT AVG(salary) FROM table_B;'"), HumanMessage(content='Hi or hello'), AIMessage(content='How can i help you sirg? '), HumanMessage(content='User question is: how are you bro. Give the sql query int the provided format.')])

In [27]:
chain = prompt|LLM |parser
chain.invoke({"DBName":"Mysql workbench","text":"how many department are there?"})

'SELECT COUNT(DISTINCT department_name) FROM table_name; \n'

## Working but  with message Placeholder

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate(
    
    [
        ("system","You are the AI assistant, answer the question in {language} language. "),
        MessagesPlaceholder(variable_name="messages"),
        ("user","user question is {input}")
    ]
)

chain = prompt|llm
chain.invoke({"messages":few_shot_examples, "language":"Marathi","input":"exaplain subodh details"})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


few_shot_dict = [
    {"question": "What is subodh height?",
     "answer": "Subodh Height is 6 feet 4 inches"},

    {"question": "Where subodh is working?",
     "answer": "Subodh is working in Wipro company"} ,

     {"question":"What technologies subodh working on?",
      "answer":"he is working on Data science"},

     ]

# Convert dictionary to list of HumanMessage and AIMessage
few_shot_examples = []
for example in few_shot_dict:
    few_shot_examples.append(HumanMessage(content=example['question']))
    few_shot_examples.append(AIMessage(content=example['answer']))

prompt = ChatPromptTemplate.from_messages([
        ("system","""You are the AI agent. 
         """),
        *few_shot_examples,
        ("user","User has asked you about: {text}. ")
    ]
) 

prompt

In [ ]:
from langchain_core.messages import AIMessage
llm.invoke([
        HumanMessage(content="Hi, I am Subodh, working as a data scientist"),
        AIMessage(content="Hi Subodh!\n\nIt's great to meet you.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people are using AI and data analysis in different fields.")
       , HumanMessage(content="Hey, What is my name and what do i do?")
    ]
)